In [20]:
import pandas as pd
import numpy as np
from yahoo_fin.stock_info import get_data
import datetime

### Defined the directory and model parameter

In [21]:
log_directory = "new_log/"
file_directory = "data/"

In [22]:
look_back = 3
portfolio_size = 10

### Extract the data from Yahoo Finance

In [23]:
with open(file_directory+"yahoo_finance_crypto_list.txt") as f:
    
    ticker_list = f.readlines()
    
    ticker_list = [ticker.strip() for ticker in ticker_list]

In [ ]:
start_date = (datetime.datetime.today()-datetime.timedelta(days=look_back)).strftime("%Y-%m-%d")
end_date = (datetime.datetime.today()).strftime("%Y-%m-%d")

In [176]:
start_date = "2021-01-14"
end_date = "2021-01-19"

In [177]:
price_dic = {}
error_list = []

for ticker in ticker_list:
    
    try:
        price_df = get_data(ticker=ticker,
                        start_date=start_date,
                        end_date=end_date,
                        index_as_date=True,
                        interval="1d")
        
        price_dic[ticker] = price_df
    
    except:
        
        print("Error.")
        error_list.append(ticker)
    
    #time.sleep(1)

Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.


In [178]:
price_df_list = [price_dic[ticker] for ticker in ticker_list if ticker not in error_list]
price_df = pd.concat(price_df_list,axis=0)

### Filter the universe by average transaction volume

In [179]:
average_volume = price_df[["volume","ticker"]].groupby("ticker").mean()
volume_threshold = average_volume.describe().loc["25%",:][0]

universe_list = list(average_volume["volume"][average_volume["volume"]>volume_threshold].index)

In [180]:
volume_threshold

28473.625

In [181]:
price_df_list = [price_dic[ticker] for ticker in universe_list]
price_df = pd.concat(price_df_list,axis=0)

### Calculate Signal

In [182]:
price_df["date"] = price_df.index
price_wide = price_df.pivot_table(values="close",columns="ticker",index="date")


signal = price_wide.pct_change(look_back)
signal = signal.iloc[-1,:].dropna()

In [183]:
signal

ticker
ABBC-USD   -0.102643
ACH-USD     0.142422
ACT-USD     0.020075
ADA-USD     0.047363
ADK-USD    -0.235803
              ...   
ZEN-USD     0.202520
ZIL-USD     0.002951
ZNN-USD    -0.139506
ZRX-USD     0.035132
ZYN-USD    -0.057019
Name: 2021-01-19 00:00:00, Length: 271, dtype: float64

### Create Portfolios

In [184]:
top_portfolio = signal.nlargest(portfolio_size)
bottom_portfolio = signal.nsmallest(portfolio_size)

top_portfolio = pd.DataFrame(top_portfolio).T
bottom_portfolio = pd.DataFrame(bottom_portfolio).T

In [185]:
bottom_portfolio

ticker,GLEEC-USD,XRC-USD,XSN-USD,CRU-USD,ZANO-USD,ADK-USD,MGO-USD,DERO-USD,NEBL-USD,AYA-USD
2021-01-19,-0.33463,-0.304936,-0.263549,-0.258402,-0.245645,-0.235803,-0.225518,-0.224443,-0.172608,-0.155057


In [186]:
top_portfolio

ticker,MTC1-USD,MTC2-USD,MAN-USD,BPS-USD,XWC-USD,PAI-USD,QRL-USD,LBC-USD,XNS2-USD,NIX-USD
2021-01-19,2.76155,2.500261,0.862814,0.802588,0.558147,0.531651,0.511923,0.506415,0.475023,0.420769


In [187]:
log_directory = "new_log/"

### Write the Log

In [188]:
end_date

'2021-01-19'

In [189]:
top_portfolio.to_csv(log_directory+"top_portfolio"+end_date+".csv")
bottom_portfolio.to_csv(log_directory+"bottom_portfolio"+end_date+".csv")

In [190]:
end_date

'2021-01-19'